In [1]:
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))

from src.extra.io import save_plot

matplotlib.rcParams['font.family'] = "monospace"
%matplotlib inline

In [ ]:
def collect_results(filename, col, algos):
    with open(filename) as file:
        results_data = json.load(file)
        
        data = [
            [model_results[algo][col] for model_results in results_data if algo in model_results] algo in algos
        ]
    return data

## Greedy algorithms

In [ ]:
greedy_filenames = []
file_type = []
algos = ['', '', '', '']

fig, axs = plt.subplots(1, 5, figsize=(14, 3.5))
for ax, filename, name in zip(axs, greedy_filenames, file_type):
    results = collect_results(f'ablibaba/greedy/{filename}', 'social welfare percent', algos)
    ax.boxplot(results, vert=False, widths=0.9)
    ax.set_xlabel('Social welfare percentage')
    ax.set_yticklabels(['', '', '', ''])
    ax.set_title(name)
    
axs[0].set_yticklabels(['Greedy\nAlgorithm', 'Fixed\nOptimal', 'Flexible\nOptimal','Server\nRelaxed'])
fig.tight_layout()
save_plot('multi_setting_social_welfare', 'greedy')

## Server resource ratio

In [ ]:
algorithms = [('Greedy Algorithm', greedy_name), ('Fixed Resource Allocation', 'Fixed Optimal')]
with open('') as file:
    results_data = json.load(file)

In [ ]:
server_fig, (ax_storage_used, ax_computation_used, ax_bandwidth_used) = plt.subplots(1, 3, figsize=(16, 4))
for ax, name, resource_func in [(ax_storage_used, 'server storage used', storage_usage), 
                 (ax_computation_used, 'server computation used', computational_usage), 
                 (ax_bandwidth_used, 'server bandwidth used', bandwidth_usage)]:
    for name, algo in algorithms:
        results = [resource_func(results_data[0][f'ratio {ratio}'][algo]) for ratio in ratios]
        ax.plot(ratios, results, label=name)
    ax.set_xticklabels(ratio_labels)
    ax.set_xticks(ratios)
    ax.set_ylabel('Resource Usage (%)')
    ax.set_xlabel('Ratio of Computation:Bandwidth')
    ax.set_title(prop)

plt.legend()
save_plot('server_resource_usage', 'resource_ratio')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
for name, algo in algorithms:
    social_welfares = [[result[f'ratio {ratio}'][algo]['social welfare'] for result in results_data] for ratio in ratios]
    error = 1.96 * np.std(social_welfares, axis=1) / np.sqrt(len(results_data))
    ax.errorbar(x=ratios, y=np.mean(social_welfares, axis=1), yerr=error, label=name)

ax.set_ylabel('Social Welfare', fontsize=12)
ax.set_yticks(fontsize=11)
ax.set_xlabel('Ratio of Computation:Bandwidth', fontsize=12)
ax.set_xticks(ticks=ratios, labels=ratio_labels, fontsize=11)

plt.legend(fontsize=12)
save_plot('social_welfare', 'resource_ratio')

## Auction algorithms

In [ ]:
auction_filenames = []
file_type = []
algos = ['', '', '', '']

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(14, 3.5))
for ax, filename, name in zip(axs, auction_filenames, file_type):
    results = collect_results(f'ablibaba/auction/{filename}', 'social welfare percent', algos)
    ax.boxplot(results, vert=False, widths=0.9)
    ax.set_xlabel('Social welfare percentage')
    ax.set_yticklabels(['', '', '', ''])
    ax.set_title(name)
    
axs[0].set_yticklabels(['', '', '', ''])
fig.tight_layout()
save_plot('multi_setting_social_welfare', 'auction')

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(14, 3.5))
for ax, filename, name in zip(axs, auction_filenames, file_type):
    results = collect_results(f'ablibaba/auction/{filename}', 'revenue', algos)
    ax.boxplot(results, vert=False, widths=0.9)
    ax.set_xlabel('Social welfare percentage')
    ax.set_yticklabels(['', '', '', ''])
    ax.set_title(name)
    
axs[0].set_yticklabels(['', '', '', ''])
fig.tight_layout()
save_plot('multi_setting_social_welfare', 'auction')

## Decentralised iterative auction heuristics

In [ ]:
with open('../data/dia_heuristics/dia_heuristic_grid_search_paper_r_t30_s6_dt07-22_12-17-27.json') as file:
    results_data = json.load(file)
    
heuristic_results = {}
for model_results in results_data:
    for heuristic, heuristic_result in model_results.items():
        if heuristic == 'model':
            continue
        elif heuristic in heuristic_results:
            heuristic_results[heuristic].append(heuristic_result)
        else:
            heuristic_results[heuristic] = [heuristic_result]

In [ ]:
heuristic_avg_results = []
for heuristic, results in heuristic_results.items():
    heuristic_avg_results.append([
        heuristic, results[0]['price change'], results[0]['initial price'],
        [r['social welfare'] for r in results],
        [r['social welfare percent'] for r in results],
        [r['percentage tasks allocated'] for r in results],
        [r['total revenue'] for r in results],
        [r['rounds'] for r in results],
        [r['solve time'] for r in results],
        [task_price for r in results for task_price in r['task prices'].values()],
        np.mean([r['social welfare'] for r in results]),
        np.mean([r['total revenue'] for r in results]),
        np.mean([r['rounds'] for r in results]),
        np.mean([task_rounds for r in results for task_rounds in r['task rounds'].values()])
    ])
    
heuristic_df = pd.DataFrame(heuristic_avg_results, columns=[
    'name', 'price change', 'initial price', 'social welfare', 'social welfare percent', 
    'percentage tasks allocated', 'revenue', 'rounds', 'solve time', 'task prices', 
    'avg social welfare', 'avg revenue', 'avg rounds', 'avg task rounds',
])
heuristic_df = heuristic_df.sort_values(by=['price change', 'initial price'])

In [ ]:
def difference(results_1, results_2):
    sw, rev, rounds, solve_time, task_rounds = [], [], [], [], []
    for r1, r2 in zip(results_1, results_2):
        sw.append(r1['social welfare'] / r2['social welfare'])
        rev.append(r1['total revenue'] / r2['total revenue'])
        rounds.append(r1['rounds'] / r2['rounds'])
        solve_time.append(r1['solve time'] / r2['solve time'])
    return np.mean(sw), np.mean(rev), np.mean(rounds), np.mean(solve_time), sw, rev, rounds, solve_time

min_heuristic_results = heuristic_results['IP: 20, PC: 1']
heuristic_difference_results = [
    [heuristic, results[0]['price change'], results[0]['initial price'], *difference(results, min_heuristic_results)]
    for heuristic, results in heuristic_results.items()
]
heuristic_diff_df = pd.DataFrame(heuristic_difference_results, columns=[
    'name', 'price change', 'initial price', 
    'social welfare diff', 'revenue diff', 'rounds diff', 'solve time diff',
    'social welfare diff list', 'revenue diff list', 'rounds diff list', 'solve time diff list'
])

In [ ]:
def heuristic_heatmap(ax, col, df, label_df, label_col, title='', print_heatmap=True,
                      initial_prices=['40', '35', '30', '25', '20'], price_changes=['1', '3', '5', '7', '10']):
    ax.set_yticks(np.arange(len(price_changes)))
    ax.set_yticklabels(list(reversed(price_changes)), fontsize=11)
    ax.set_ylabel('Bid Increment', fontsize=13)
    
    ax.set_xticks(np.arange(len(initial_prices)))
    ax.set_xticklabels(list(reversed(initial_prices)), fontsize=11)
    ax.set_xlabel('Reserve Price', fontsize=13)
    
    heatmap = ax.imshow(df.pivot(index='price change', columns='initial price', values=col).sort_index(axis=0, ascending=False))
    colour_bar = plt.colorbar(heatmap, shrink=0.82, ax=ax)
    ax.set_title(title)
    
    for x, initial_price in enumerate(initial_prices):
        for y, price_change in enumerate(price_changes):
            label = label_df[label_df['name'] == f'IP: {initial_price}, PC: {price_change}'].get(label_col).values[0]
            ax.text(len(initial_prices) - 1 - x, len(price_changes) - 1 - y, 
                    round(label), color='white', ha='center', va='center', weight='bold', fontsize=12)
            
    if print_heatmap:
        print(f'\n\t{col}')
        print(df.pivot(index='price change', columns='initial price', 
                       values=col).sort_index(axis=0, ascending=False))
        print(f'\n\t{label_col}')
        print(label_df.pivot(index='price change', columns='initial price', 
                             values=label_col).sort_index(axis=0, ascending=False))
        print()

In [ ]:
fig, (ax_social_welfare, ax_revenue, ax_rounds) = plt.subplots(1, 3, figsize=(15, 4.5))

heuristic_heatmap(ax_social_welfare, 'social welfare diff', heuristic_diff_df, heuristic_df, 'avg social welfare', 'Social Welfare')
heuristic_heatmap(ax_revenue, 'revenue diff', heuristic_diff_df, heuristic_df, 'avg revenue', 'Revenue')
heuristic_heatmap(ax_rounds, 'rounds diff', heuristic_diff_df, heuristic_df, 'avg rounds', 'Rounds')

plt.tight_layout()
save_plot('grid_search', 'dia_heuristics')

## Decentralised iterative auction mutation

## Online versions